# Introduction à pandas

[pandas](https://pandas.pydata.org/) propose une structure de données à base de tableaux et des opérations associées optimisée pour l'analyse de données en Python. Cette structure analogue aux *data frames* du langage R est codée en Python autour des structures numpy.

Les possibilités de `pandas` sont immenses et on trouvera de nombreuses introductions exhaustives en ligne. [(1)](http://pandas.pydata.org/pandas-docs/stable/tutorials.html) [(2)](https://bitbucket.org/hrojas/learn-pandas) [(3)](https://www.dataquest.io/blog/pandas-python-tutorial/)

Une fois l'outil compris, les indispensables :
 - la documentation officielle de [pandas](http://pandas.pydata.org/pandas-docs/stable/);
 - cette feuille récapitulative [ici](https://github.com/kailashahirwar/cheatsheets-ai/raw/master/PDFs/Pandas.pdf).

Nous avons pris le parti ici de présenter cet outil autour d'une petite étude de cas. Dans le cadre du programme d'[ouverture des données](https://www.data.gouv.fr/fr/), le gouvernement impose à tous les distributeurs de carburants de publier leurs tarifs à chaque mise à jour. On trouvera alors [ici](https://www.prix-carburants.gouv.fr/) une interface qui donne accès à ces données, également téléchargeable sour forme d'archive (au format XML).

Nous avons alors transformé ce fichier XML en deux fichiers CSV, un contenant les [mises à jour de tarifs](02-prix2016.csv) par station et par type de carburant (année 2016), et un autre fichier qui donne quelques informations (notamment l'adresse) de chaque station.



## Les bases de l'outil

In [ ]:
import pandas as pd

Le fichier étant très gros, on commence par se limiter aux premières lignes pour vérifier que la lecture du fichier se passe bien :

In [ ]:
carburant = pd.read_csv("data/02-prix2016.csv", nrows=8)
carburant

Les données sont réparties en colonnes, auxquelles on peut accéder par leur nom:
 - soit par la notation pointée (à condition que le nom de la colonne ne soit pas un mot-clé du langage Python);
 - soit, dans le cas général, par la notation en crochet.

In [ ]:
carburant.id

In [ ]:
carburant['maj']

Le type de chacune de ces colonnes ressemble à un tableau NumPy. En réalité, il s'agit d'une *série* Pandas, qui hérite (probablement) d'un tableau NumPy. Chaque série a notamment un *nom* et un *type* (NumPy).

On peut récupérer le tableau NumPy sous-jacent à une série pandas par l'attribut `values` :

In [ ]:
carburant['valeur'].values

Les séries restent des itérables que l'on peut utiliser pour construire d'autres structure de données si besoin :

In [ ]:
set(carburant.id)

<div class='alert alert-danger'>**Petit souci:** Les dates sont encodées comme des objets Python (en réalité des chaînes de caractères)</div>

On souhaite les interpréter comme des dates. La fonction `read_csv` permet cela :

In [ ]:
type(carburant['maj'].values[0])

In [ ]:
carburant = pd.read_csv("data/02-prix2016.csv", parse_dates=['maj'], nrows=10)
carburant

In [ ]:
carburant['maj'].dtype  # Cette réponse cryptique signifie np.datetime64[ns] (ns pour nanoseconds)

<div class='alert alert-warning'>
**À vous de jouer !** Tout est prêt pour la suite: on peut décoder le fichier entièrement.
</div>

    ☞   Utiliser la méthode .head() pour n'afficher que le début du tableau

In [ ]:

carburant.head()

In [ ]:
carburant.shape

<div class='alert alert-warning'>
**À vous de jouer !** On souhaite maintenant renommer les colonnes.
</div>

    Pour la suite du notebook:
    ☞   `maj` deviendra `date`;
    ☞   `nom` deviendra `type`;
    ☞   `valeur` deviendra `prix`.

In [ ]:
carburant.columns

In [ ]:
carburant.columns = ['id date type prix'.split()]

## Les opérations de base

Outre les opérations habituelles sur les tableaux `numpy`, les séries `pandas` sont munies de nombreuses opérations assez intuitives, notamment `min()`, `max()`, `mean()`, `count()`, `std()`, etc.

<div class='alert alert-warning'>
**À vous de jouer !** Quel est le prix de carburant le moins cher de notre tableau de données?
</div>


Ici, on ne récupère que la valeur de ce minimum.

<div class='alert alert-warning'>
Modifier alors la ligne suivante pour **filtrer** les lignes pour lesquelles le prix est égal à ce minimum.
</div>

In [ ]:
carburant[carburant['prix'] == 1.653]

Comme la condition écrite sur le tableau renvoie un tableau de booléens (donc *itérable* !), on peut facilement compter le nombre de lignes qui vérifient cette condition.

In [ ]:
sum(carburant['prix'] == 0.5)

Nous sommes tout de même face à un problème: le fichier contient manifestement des incohérences. On va pouvoir filtrer les lignes qui ont un tarif qui nous semble anormalement bas, mais on ne sait pas vraiment quelle borne choisir.

### Matplotlib intégré

Les structures de données pandas sont toutes munies d'opérations d'affichage sous `matplotlib`. La syntaxe de chacune des méthodes est la même que dans le matplotlib classique, on peut même ajouter le paramètre `ax` pour pouvoir contrôler plus finement le placement des graphes.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
carburant['prix'].plot.hist(bins=20, ax=ax)

## Le mot-clé `groupby`

La distribution que nous voyons apparaître ne semble pas être naturelle (= gaussienne).  
Nous allons essayer de préciser un peu la distribution des prix.

`pandas` propose l'opération `groupby` qui s'applique bien aux colonnes qui représentent une catégorie (ici le type de carburant).

In [ ]:
carburant.groupby('type')

In [ ]:
carburant.groupby('type').get_group('SP98').head(5)

On pourra alors construire autant de sous-tableaux que de catégories.

<div class='alert alert-success'>
**Note**: Dans la ligne précédente, nous avons *chaîné* trois opérations qui chacune renvoyaient une vue ou un nouveau tableau pandas. La « philosophie pandas » se retrouve dans ces chaînages d'opérations qui peuvent atteindre 10 ou 20 opérations...
</div>

Pour notre besoin ici, on va profiter du fait que le type `DataFrameGroupBy` est itérable pour enchaîner élégamment les opérations :

In [ ]:
groups = carburant.groupby('type')['prix']  # on accède à la colonne `prix` pour toutes les catégories

fig, ax = plt.subplots(figsize=(7, 5))

for key, value in groups:
    value.hist(label=key, bins=20, alpha=.5, ax=ax)

ax.legend()

<div class='alert alert-warning'>
**À vous de jouer !** Filtrer les lignes du tableau pour lesquelles le prix du carburant est inférieur à 0,4 €/L
</div>

Pour aller un peu plus loin avec le mot-clé `groupby`:
 - on peut facilement enchaîner un `groupby` avec une opération standard (moyenne, nombre d'occurrences, etc.) ;
 - on peut appliquer un `groupby` sur plusieurs colonnes (*on y vient...*)

In [ ]:
# ['prix'] renvoie une série
# [['prix']] renvoie un tableau à une colonne (avec pretty-print)
carburant.groupby('type')[['prix']].mean()

<div class='alert alert-warning'>
**À vous de jouer !** Compter le nombre de lignes de mise à jour de prix par type de carburant.
</div>

## Ajout de nouvelles colonnes

Il existe plusieurs méthodes pour créer ou remplacer le contenu de colonnes d'un tableau.  
La méthode la plus sûre qui fonctionne toujours sans messages d'erreur ou d'avertissement est basée sur `assign`.

<div class='alert alert-warning'>
**À vous de jouer !** Consulter l'aide de la méthode `assign`.
</div>

<div class='alert alert-warning'>
**À vous de jouer !** Ajouter une colonne `month` qui donne le mois courant de la date mise à jour du tarif du carburant.
</div>

    ☞   Utiliser l'attribut month des datetime Python
    ☞   Éventuellement, rechercher de l'aide sur la méthode apply

On reprend alors l'étude précédente mais on souhaite trouver le prix moyen par mois et par type de carburant.  
On passe alors deux arguments au `groupby`.

In [ ]:
carburant.groupby(('type', 'month'))['prix'].mean()

## Pivot ligne/colonne

Cette présentation de tableau étant peu lisible, `pandas` offre une opération qui transforme ce tableau 1D à deux index en tableau 2D. On choisit alors de passer le type de carburant en colonne :

In [ ]:
carburant.groupby(('type', 'month'))['prix'].mean().unstack('type')

On peut alors facilement tracer cette l'évolution du prix du carburant en 2016, par type de carburant :

In [ ]:
carburant.groupby(('type', 'month'))['prix'].mean().unstack('type').plot.line()

<div class='alert alert-warning'>
**À vous de jouer !** L'évolution du prix sur le graphe précédent étant un peu grossière, tracer le même graphe avec un point par semaine.
</div>

    ☞   Utiliser l'attribut week des datetime Python
    ☞   Pour éviter les recouvrements, donner le no. de semaine 0 aux jours de janvier de la semaine 53
    
<div class='alert alert-danger'>
**Avertissement** : si vous voyez un message sur fond rouge, lisez-le et faites bêtement ce qu'on vous demande. On reviendra dessus plus loin.
</div>

In [ ]:
carburant.groupby(('type', 'week'))['prix'].mean().unstack('type').plot.line()

## Analyse en composantes principales

Poursuivons notre analyse de l'évolution des prix du carburant en nous concentrons sur un type de carburant en particulier.

Une des applications de l'analyse en composantes principales (PCA pour *Principal Component Analysis*) permet de projeter un vecteur de grande dimension sur une nombre de dimensions plus petit, sur lesquelles s'exprime la plus grande variance.

C'est un excellent outil de base pour visualiser d'éventuelles classes de comportements.

<div class='alert alert-warning'>
**À vous de jouer !** Sélectionner les lignes du tableau correspondant au carburant SP98.
</div>

<div class='alert alert-warning'>
**À vous de jouer !** Créer un tableau du prix moyen du SP98 par semaine (ligne) et par station (colonne).
</div>

In [ ]:
from sklearn.decomposition import PCA

try:
    Xpca = PCA(n_components=2).fit_transform(stats.T)
except Exception as e:  # No real need to catch this, but the output is really long
    print('{}: {}'.format(type(e).__name__, e))

<div class='alert alert-warning'>
**À vous de jouer !** La PCA ne fonctionne pas pour des vecteurs contenant des `NaN`. Comment pourrait-on remplacer ces `NaN` par des valeurs ? Trouver la méthode appropriée dans la documentation de pandas et relancer l'analyse en composantes principales.
</div>

<div class='alert alert-success'>
Sur cette représentation, on devine un nuage de points à deux bosses, mais les points qui s'alignent en haut de l'image semblent étranges.
</div>

On peut sélectionner grossièrement les indices des points pour lesquels $y < -0.2$

In [ ]:
import numpy as np
idx_y = np.where(Xpca[:, 1] < -.2)[0]
idx_y

On peut alors afficher le tableau `stats` construit précédemment uniquement pour les stations sélectionnées. Le mot-clé `iloc` permet d'indexer un tableau `pandas` comme on aurait indexer un tableau `numpy`.

<div class='alert alert-warning'>
**À vous de jouer !** Afficher les premières lignes du tableau stats pour les stations *atypiques*. Proposer une interprétation pour ces stations qui ont été séparées du nuage de points.
</div>

In [ ]:
fig, (ax0, ax1) = plt.subplots(2, 1, figsize=(7, 10), sharex=True)

stats.iloc[:, idx_y].isnull().sum().plot.hist(ax=ax0)
stats.isnull().sum().plot.hist(ax=ax1)

ax0.set_title("Stations filtrées")
ax1.set_title("Toutes les stations")

fig.suptitle("Nombre de semaines à prix = NaN")

<div class='alert alert-success'>
On peut alors choisir de filtrer les stations qui n'ont plus de 10 semaines sans mise à jour de tarif.
</div>

In [ ]:
stats_filtered = stats_filled.loc[:,stats.isnull().sum() < 10]
Xpca = PCA(n_components=2).fit_transform(stats_filtered.T)

fig, ax = plt.subplots(figsize=(7, 5))
ax.scatter(Xpca[:, 0], Xpca[:, 1], s=5)

<div class='alert alert-success'>
On peut maintenant essayer d'interpréter ces nuages de points en utilisant la couleur pour afficher une troisième dimension.
</div>

Le paramètre `cmap` prend le nom d'une [`colormap`](http://matplotlib.org/users/colormaps.html) (suivre le lien pour une liste de colormap par défaut).

In [ ]:
fig, ax = plt.subplots(figsize=(8.8, 5))
s = ax.scatter(Xpca[:, 0], Xpca[:, 1], s=5, c=stats_filtered.mean(), cmap="YlOrRd")
fig.colorbar(s, label='Prix moyen par station')

## Jointures

Essayons à présent d'exploiter les informations présentes dans le second fichier.

In [ ]:
stations = pd.read_csv('data/02-stations2016.csv')
stations.head()

Chaque ligne reprennent l'identifiant d'une station, commun avec le 1er fichier, puis donne l'adresse, le code postal (lu comme un entier) et la ville du fichier. La dernière colonne vaut `A` si la station service est sur une aire d'autoroute.

Pour manipuler les codes postaux, commençons par les écrire sous forme de chaîne de caractère de cinq chiffres (complétée par des zéros en tête).

In [ ]:
stations = stations.assign(
    code_postal = stations.cp.apply(lambda x: "{:0>5}".format(x)))
stations.head()

<div class='alert alert-warning'>
**À vous de jouer !** Ajouter une colonne département avec les deux premiers chiffres du code postal (trois pour les DOM écrits sous la forme 97x).
</div>

<div class='alert alert-success'>
On souhaite reprendre le nuage de points précédent pour afficher dans une couleur différente toutes les stations services présentes sur autoroute.  
*☞&emsp;On cherche une interprétation des nuages de points basée sur le fait qu'une station est sur autoroute ou non*.
</div>

Il nous faut alors construire un vecteur issu de `stations['pop']` pour les stations présentes dans le nuage de points.  
Souvenez-vous qu'on a enlevé des stations "aberrantes"...

Voici une manière de faire :
- Nous avons vu ci-dessus l'opérateur `.iloc[]` pour sélectionner des cellules en fonction de leur position ;
- L'opérateur `.loc[]` fonctionne de la même manière avec l'**index** des lignes et le **nom** des colonnes.

Commençons alors par réindexer le tableau `stations` sur les identifiants :

In [ ]:
stations.set_index('id').head()

On va pouvoir alors l'associer aux colonnes de notre tableau `stats_filtered` :

In [ ]:
stats_filtered.columns

In [ ]:
# Observer la colonne `id` pour constater qu'on a bien filtré des lignes...
stations.set_index('id').loc[stats_filtered.columns].head()

On peut alors reprendre le nuage de points avec le paramètre `c` qui prend ici un vecteur de booléens :

In [ ]:
fig, ax = plt.subplots(figsize=(7, 5))
s = ax.scatter(Xpca[:, 0], Xpca[:, 1], s=5, cmap='Paired',
               c=(stations.set_index('id').loc[stats_filtered.columns]['pop'] == 'A'))
fig.suptitle('Stations sur autoroute')

<div class='alert alert-danger'>
**Attention :** aucune corrélation claire se dessine sur ce graphe. On ne peut donc rien conclure de plus avec ce graphe, (à part que les stations services sur autoroute sont parmi les plus chères).
</div>

**Essayons autrement** : reprenons notre analyse en composantes principales en faisant une moyenne des prix des stations services sur un département.

Problème : les prix sont dans une table, les départements dans une autre. On souhaiterait rajouter une colonne dans notre tableau `sp98` avec le numéro du département de chaque station.

Une opération permet ceci : la **jointure** des bases de données est proposée sous la fonction `merge`. ([documentation](https://pandas.pydata.org/pandas-docs/stable/merging.html))  
Elle prend notamment en paramètre le nom de la colonne sur laquelle se baser pour fusionner les deux tableaux.

In [ ]:
sp98_dept = sp98.merge(stations, on='id')
sp98_dept.head()

<div class='alert alert-warning'>
**À vous de jouer !** Faire une nouvelle analyse par composantes principales avec un point par département.
</div>

<div class='alert alert-success'>
**Analyse** – Quelques départements se démarquent du nuage de point : Paris (75), les trois départements 32, 82 et 92 (dans une moindre mesure) et la Réunion (974) pour une raison probablement différente.
</div>

Essayons de comprendre pourquoi :

In [ ]:
fig, ax = plt.subplots(figsize=(7, 5))
stats_dept.mean(axis=1).plot(label='France', ax=ax)
stats_dept['75 92 17 974'.split()].plot(ax=ax)
plt.legend()

<div class="alert alert-warning">
**À vous de jouer !** On devine que l'axe des abcisses est encore une fois corrélé au prix moyen par département.
<br /> Essayer une interprétation pour l'axe des ordonnées.
</div>

In [ ]:
fig, ax = plt.subplots(figsize=(8.8, 5))
points = plt.scatter(*Xpca.T, c=stats_dept.mean(), cmap='YlOrRd')
for (x, y), name in zip(Xpca, stats_dept.columns):
    plt.annotate(name, (x,y))
plt.colorbar(label='Prix moyen par station')



<div class="alert alert-danger">
**Attention ** au biais que peut causer le choix des couleurs sur l'interprétation !
</div>

## Un petit dernier pour la route

<div class='alert alert-warning'>
**À vous de jouer !** Voici une carte de France.<br />
Colorer chaque département en fonction du prix moyen du carburant dans les stations services présentes sur son territoire.
</div>

In [ ]:
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader

lambert93 = ccrs.LambertConformal(3, 46.5, standard_parallels=(44, 49),
                                  false_easting=700000, false_northing=6600000)
fig = plt.figure(figsize=(15, 10))
ax = plt.axes(projection=lambert93)

admin1_file = shpreader.natural_earth(resolution='10m', category='cultural',
                                      name='admin_1_states_provinces')

shapes = {r.attributes['gn_a1_code'][3:]: r.geometry
          for r in shpreader.Reader(admin1_file).records()
          if r.attributes['adm0_a3'] == 'FRA'
          and r.attributes['gn_a1_code'][:2] == 'FR'}

for dept in shapes:
    ax.add_geometries(shapes[dept], ccrs.PlateCarree(), 
                      edgecolor="#aaaaaa", facecolor="yellow")
    
ax.coastlines('10m', color="#226666")

ax.set_xlim((80000, 1150000))
ax.set_ylim((6100000, 7150000))